# Recreating https://arxiv.org/pdf/1608.02609.pdf first for $\rho$-$\rho$ decays

## Inputs:
- Invariant masses of intermediate resonances  
- Acoplanarity angles
- Variables $y_i^+$ ($y_k^-$)
- 4 momentum of visible decay products
- 4 momentum of intermediate resonances
    - If cascade decays, need to provide 4-momenta of all $\pi^+\pi^-$ pairs which can form the resonances
- Need to boost all four vectors where primary resonances are aligned along the z-axis.
- Normalise all features

In [4]:
import uproot 
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from pylorentz import Momentum4
from pylorentz import Position4
from lbn_modified import LBN, LBNLayer
from ROOT import TLorentzVector, TVector3

In [11]:
tree_tt = uproot.open("/eos/user/s/stcheung/SWAN_projects/Masters_CP/MVAFILE_AllHiggs_tt.root")["ntuple"]

variables = [
            "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand",
            "aco_angle_1", 
            "mva_dm_1","mva_dm_2",
            "tau_decay_mode_1","tau_decay_mode_2",
        #     "ip_x_1", "ip_y_1", "ip_z_1", "ip_x_2", "ip_y_2", "ip_z_2", # ignore impact parameter for now
            "pi_E_1", "pi_px_1", "pi_py_1", "pi_pz_1", 
            "pi_E_2", "pi_px_2", "pi_py_2", "pi_pz_2", 
            "pi0_E_1", "pi0_px_1", "pi0_py_1", "pi0_pz_1",
            "pi0_E_2", "pi0_px_2", "pi0_py_2", "pi0_pz_2", 
            "y_1_1", "y_1_2"
        ]
df = tree_tt.pandas.df(variables)
# select only rho-rho events
df = df[(df['mva_dm_1']==1) & (df['mva_dm_2']==1) & (df["tau_decay_mode_1"] == 1) & (df["tau_decay_mode_2"] == 1)]
# drop unnecessary labels 
df = df.drop(["mva_dm_1","mva_dm_2","tau_decay_mode_1","tau_decay_mode_2", "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand"], axis=1).reset_index(drop=True)

In [12]:
df.head()

,aco_angle_1,pi_E_1,pi_px_1,pi_py_1,pi_pz_1,pi_E_2,pi_px_2,pi_py_2,pi_pz_2,pi0_E_1,pi0_px_1,pi0_py_1,pi0_pz_1,pi0_E_2,pi0_px_2,pi0_py_2,pi0_pz_2,y_1_1,y_1_2
0,2.657029,45.423448,-13.747046,-38.825621,-19.153590,35.805782,8.526798,34.653880,2.904638,10.039846,-3.551914,-8.385803,-4.224041,6.112238,1.959179,5.735656,0.777900,-0.637964,-0.708372
1,1.547246,24.506373,-14.056253,9.809321,-17.514046,19.587723,8.539313,-3.149378,-17.344192,41.244844,-24.443820,15.370633,-29.451014,77.036002,32.655902,-12.032904,-68.726477,0.254573,0.594557
2,3.750274,15.319610,8.794122,0.774085,12.519393,17.469988,-9.876231,-3.253855,14.037575,69.457213,38.899682,2.503120,57.487698,56.810782,-32.919951,-10.831700,45.015529,0.638590,0.529623
3,1.192612,94.211361,42.942276,-33.711103,-76.780750,3.656937,-2.129749,0.251363,-2.958835,25.585962,11.770386,-8.993712,-20.861314,68.839211,-41.770362,4.326843,-54.546623,-0.572846,0.899114
4,3.264792,25.899289,-22.245884,-12.141924,-5.333670,23.795390,21.402816,9.436143,-4.368043,23.216735,-20.268636,-10.296444,-4.707891,18.242053,16.607666,6.693405,-3.483969,-0.054617,-0.132105


- [x] calculate 4 momentum of resonances
- [x] calculate invariant mass of resonances m^2 = E^2 - p^2
- [ ] cascades?

In [35]:
# parse particle information
columns = ['E', 'px', 'py', 'pz']

pi_1 = df[['pi_E_1', "pi_px_1", "pi_py_1", "pi_pz_1"]]
pi_2 = df[['pi_E_2', "pi_px_2", "pi_py_2", "pi_pz_2"]]
pi0_1 = df[['pi0_E_1', "pi0_px_1", "pi0_py_1", "pi0_pz_1"]]
pi0_2 = df[['pi0_E_2', "pi0_px_2", "pi0_py_2", "pi0_pz_2"]]

pi_1.columns = columns
pi_2.columns = columns
pi0_1.columns = columns
pi0_2.columns = columns

In [34]:
# reconstruct resonance 4 momentum
rho_1 = pi_1 + pi0_1
rho_2 = pi_2 + pi0_2

In [33]:
# calculate invariate masses
rho_1['m'] = rho_1['E']**2 - rho_1['px']**2 - rho_1['py']**2 - rho_1['pz']**2
rho_2['m'] = rho_2['E']**2 - rho_2['px']**2 - rho_2['py']**2 - rho_2['pz']**2

In [7]:
df.head()

,pi_E_1,pi_px_1,pi_py_1,pi_pz_1,pi_E_2,pi_px_2,pi_py_2,pi_pz_2,pi0_E_1,pi0_px_1,...,pi0_px_2,pi0_py_2,pi0_pz_2,y_1_1,y_1_2,aco_angle_1,tau_decay_mode_1,tau_decay_mode_2,mva_dm_1,mva_dm_2
entry,,,,,,,,,,,,,,,,,,,,,
8,45.423448,-13.747046,-38.825621,-19.153590,35.805782,8.526798,34.653880,2.904638,10.039846,-3.551914,...,1.959179,5.735656,0.777900,-0.637964,-0.708372,2.657029,1,1,1,1
25,24.506373,-14.056253,9.809321,-17.514046,19.587723,8.539313,-3.149378,-17.344192,41.244844,-24.443820,...,32.655902,-12.032904,-68.726477,0.254573,0.594557,1.547246,1,1,1,1
27,15.319610,8.794122,0.774085,12.519393,17.469988,-9.876231,-3.253855,14.037575,69.457213,38.899682,...,-32.919951,-10.831700,45.015529,0.638590,0.529623,3.750274,1,1,1,1
45,94.211361,42.942276,-33.711103,-76.780750,3.656937,-2.129749,0.251363,-2.958835,25.585962,11.770386,...,-41.770362,4.326843,-54.546623,-0.572846,0.899114,1.192612,1,1,1,1
50,25.899289,-22.245884,-12.141924,-5.333670,23.795390,21.402816,9.436143,-4.368043,23.216735,-20.268636,...,16.607666,6.693405,-3.483969,-0.054617,-0.132105,3.264792,1,1,1,1
